In [2]:
import os
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer,PolynomialFeatures,StandardScaler,MinMaxScaler



from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# MODELS
from sklearn.neighbors import NearestNeighbors

In [3]:
def mean(list):
    return sum(list)/len(list)

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [4]:
# COLLABORATIVE RECS -- Future application; need users

### Applying weightings to homes that have been upvoted by users with similar profile characteristics

### Strength of weighting should be inversely related to how close user input is to the user input that liked that house
### Potentially use deviations of scaler to determine how far a value is away from norm

# Python --> Crab library

In [5]:
# User Input Considerations:

# I'm Feeling Lucky -> Drop a random dot in the housing map

## Price Range: Min - Max

## Property Type

# Minimum Requirements ^^

## State

## City

## Zip Code

## No. of Bedrooms

## No. of Bathrooms

## Square Footage

## Year Built



### NEED TO IDENTIFY WHEN DATA IS SPARSE FOR A CERTAIN AREA AND PRESENT APOLOGY MESSAGE


In [295]:
aws_bucket=os.getenv('AWS_RE_BUCKET')
aws_key=os.getenv('AWS_RE_BUCKET_KEY')
aws_secret=os.getenv('AWS_RE_BUCKET_SECRET')

date="2023-06-16"
state="CA"
states=['AL',
 'NE',
 'AK',
 'NV',
 'AZ',
 'NH',
 'AR',
 'NJ',
 'CA',
 'NM',
 'CO',
 'NY',
 'CT',
 'NC',
 'DE',
 'ND',
 'DC',
 'OH',
 'FL',
 'OK',
 'GA',
 'OR',
 'HI',
 'PA',
 'ID',
 'PR',
 'IL',
 'RI',
 'IN',
 'SC',
 'IA',
 'SD',
 'KS',
 'TN',
 'KY',
 'TX',
 'LA',
 'UT',
 'ME',
 'VT',
 'MD',
 'VA',
 'MA',
 'VI',
 'MI',
 'WA',
 'MN',
 'WV',
 'MS',
 'WI',
 'MO',
 'WY',
 'MT',
]

In [296]:
# PULLING IN DATA

df_list=[]
for st in states:
    try:
        df = pd.read_parquet(path=aws_bucket+f"/{date}/{st}.parquet",storage_options={"key":aws_key,"secret":aws_secret})
        df_list.append(df)
    except:
        pass
    
df=pd.concat(df_list)

In [304]:
df['PROPERTY TYPE'].unique()

array(['Mobile/Manufactured Home', 'Vacant Land',
       'Single Family Residential', 'Multi-Family (2-4 Unit)',
       'Multi-Family (5+ Unit)', 'Townhouse', 'Ranch', 'Condo/Co-op',
       'Other', 'Parking', 'Timeshare', 'Moorage', 'Unknown', 'None'],
      dtype=object)

In [297]:
df_copy=deepcopy(df)

In [298]:
df_copy.columns

Index(['SALE TYPE', 'TimeStamp', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS',
       'CITY', 'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS',
       'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
       'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [302]:
df=df_copy

In [303]:
df

,SALE TYPE,TimeStamp,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,2023-06-16 13:20:28,nan,Mobile/Manufactured Home,4900 County Road 15,Maplesville,AL,36750,179000.0,4.0,...,Active,,,https://www.redfin.com/AL/Maplesville/4900-Cou...,Greater Alabama MLS,1356836,N,Y,32.7525199,-86.797908
1,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,0 County Road 351 #1,Maplesville,AL,36750,65000.0,nan,...,Active,,,https://www.redfin.com/AL/Maplesville/0-County...,Greater Alabama MLS,1356603,N,Y,32.7551485,-86.7817719
2,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,2523 County Road 359 #2,Maplesville,AL,36750,49900.0,nan,...,Active,,,https://www.redfin.com/AL/Maplesville/2523-Cou...,Greater Alabama MLS,1356263,N,Y,32.7038261,-86.784188
3,MLS Listing,2023-06-16 13:20:28,nan,Single Family Residential,9455 Highway 191,Maplesville,AL,36750,650000.0,5.0,...,Active,,,https://www.redfin.com/AL/Maplesville/9455-AL-...,Greater Alabama MLS,1355967,N,Y,32.8606448,-86.8135036
4,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,o Wooley Springs Rd,Athens,AL,35613,525000.0,nan,...,Active,,,https://www.redfin.com/AL/Athens/o-Wooley-Spri...,VMLS,1834445,N,Y,32.792574,-86.828275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6836,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,"201 Bear Paw Court, Whitefish",Other-see Remarks,MT,59937,269000.0,nan,...,Active,,,https://www.redfin.com/MT/Whitefish/201-Bear-P...,BMTMLS,329123,N,Y,48.33853256,-114.33277402
6837,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,"22 Marina Crest Lane, Whitefish",Other-see Remarks,MT,59937,889000.0,nan,...,Active,,,https://www.redfin.com/MT/Whitefish/22-Marina-...,BMTMLS,337088,N,Y,48.423595,-114.348704
6838,MLS Listing,2023-06-16 13:20:28,nan,Vacant Land,Nhn Cliff Vw,Whitefish,MT,59937,700000.0,nan,...,Active,,,https://www.redfin.com/MT/Whitefish/Nhn-Cliffv...,MRMLS,22118124,N,Y,48.430286,-114.37436
6839,MLS Listing,2023-06-16 13:20:28,nan,Townhouse,247 B Elm Ct,Whitefish,MT,59937,669900.0,2.0,...,Active,,,https://www.redfin.com/MT/Whitefish/247-Elm-Ct...,MRMLS,22201866,N,Y,48.385122,-114.330828


In [305]:
# PREPROCESSING
prop_types=['Single Family Residential', 'Mobile/Manufactured Home','Townhouse', 'Multi-Family (2-4 Unit)', 'Condo/Co-op','Multi-Family (5+ Unit)']

df=df.applymap(lambda x: np.nan if x=='nan' else x)

#zip_count = df[['ZIP OR POSTAL CODE']].nunique().iloc[0]

#df=df[(df['CITY']=='Houston')&(df['STATE OR PROVINCE']=='TX')] # TEMP LIMITER 1

#df=df[df['PROPERTY TYPE']=='Single Family Residential'] # TEMP LIMITER 2

df=df[df['PROPERTY TYPE'].isin(prop_types)]

df=df.dropna(subset=['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)'],axis=0).reset_index(drop=True).apply(lambda row: pd.to_numeric(row,errors='ignore'))


df_full=df

df_final=df_full.drop(['ADDRESS','CITY','STATE OR PROVINCE','ZIP OR POSTAL CODE','PROPERTY TYPE'],axis=1)


In [306]:
df.sample(frac=1, replace=False, random_state=42)

,SALE TYPE,TimeStamp,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
320111,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,893 Fiddler Creek Way,Lexington,KY,40515,599900.0,5.0,...,Active,,,https://www.redfin.com/KY/Lexington/893-Fiddle...,LBARMLS,23010381,N,Y,37.960447,-84.503518
34239,MLS Listing,2023-06-16 13:20:28,NaN,Townhouse,2172 W Million Ln,Fayetteville,AR,72704,339900.0,3.0,...,Active,,,https://www.redfin.com/AR/Fayetteville/2172-W-...,NWARMLS,1242679,N,Y,36.098520,-94.189602
105283,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,2910 Bromwich Ct,Fayetteville,NC,28306,435000.0,3.0,...,Active,,,https://www.redfin.com/NC/Fayetteville/2910-Br...,LPRMLS,705644,N,Y,34.994953,-79.012698
223706,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,96382 E Paradise Rd,Gore,OK,74435,475000.0,3.0,...,Active,,,https://www.redfin.com/OK/Gore/96382-Paradise-...,MLSOK,1058360,N,Y,35.625321,-95.068388
70282,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,28 Briarwood Pl,Roswell,NM,88201,330000.0,4.0,...,Active,,,https://www.redfin.com/NM/Roswell/28-Briarwood...,NMMLS,20232184,N,Y,33.421485,-104.484347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,8640 S Blackstone Ave,Chicago,IL,60619,330000.0,5.0,...,Active,,,https://www.redfin.com/IL/Chicago/8640-S-Black...,MRED,11735621,N,Y,41.737483,-87.588132
365838,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,2601 Oak Dr,Amarillo,TX,79107,155000.0,3.0,...,Active,,,https://www.redfin.com/TX/Amarillo/2601-Oak-Dr...,AARMLS,23-59,N,Y,35.239798,-101.828534
131932,MLS Listing,2023-06-16 13:20:28,NaN,Single Family Residential,1227 W 25th,Lorain,OH,44052,119900.0,4.0,...,Active,,,https://www.redfin.com/OH/Lorain/1227-W-25th-P...,Yes-MLS,4454790,N,Y,41.447207,-82.186509
146867,MLS Listing,2023-06-16 13:20:28,NaN,Townhouse,205 N Franklin & E Fortune St Unit A2,Tampa,FL,33602,1850000.0,3.0,...,Active,,,https://www.redfin.com/FL/Tampa/205-N-Franklin...,Stellar MLS,T3414052,N,Y,27.946667,-82.457210


In [292]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490988 entries, 0 to 490987
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ADDRESS             490988 non-null  object 
 1   CITY                490988 non-null  object 
 2   STATE OR PROVINCE   490988 non-null  object 
 3   ZIP OR POSTAL CODE  490988 non-null  object 
 4   PROPERTY TYPE       490988 non-null  object 
 5   PRICE               490988 non-null  float64
 6   BEDS                490988 non-null  float64
 7   BATHS               490988 non-null  float64
 8   SQUARE FEET         490988 non-null  float64
 9   YEAR BUILT          490988 non-null  float64
 10  DAYS ON MARKET      490988 non-null  float64
 11  LATITUDE            490988 non-null  float64
 12  LONGITUDE           490988 non-null  float64
dtypes: float64(8), object(5)
memory usage: 48.7+ MB


In [315]:
df_NN=df_final[['BEDS','BATHS','SQUARE FEET','YEAR BUILT']]

In [316]:
# Define the transformations for each column
preprocessor = ColumnTransformer(
transformers=[
    ('minmax_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET'])
])

# Apply the transformations in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

df_NN=pipeline.fit_transform(df_NN)

### Need to reduce impact of Year Built, so will probably require custom distance func

In [317]:
input_df = pd.DataFrame({'BEDS':[2],'BATHS':[2],'SQUARE FEET':[1800]})
input_df

input_t=pipeline.transform(input_df)
input_t


array([[2.02020202e-02, 3.00000000e-04, 8.64817406e-05]])

In [318]:
input_t[0][0]

0.020202020202020204

In [319]:
from scipy.spatial.distance import euclidean
from sklearn.neighbors import DistanceMetric

def weighted_euclidean(x,y):
    weights=np.array([1,1,1])
    
    return euclidean(x,y,weights)


# NEED TO FIGURE OUT HOW TO APPLY WEIGHTS PROPERLY


In [320]:

knn=NearestNeighbors(n_neighbors=df_NN.shape[0],metric=weighted_euclidean)
knn.fit(df_NN)

distances, indices = knn.kneighbors(input_t)

        
        

    

In [325]:
indices[0][1]

367240

In [329]:
df_final.loc[indices[0][0:5]]

,SALE TYPE,TimeStamp,SOLD DATE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
141908,MLS Listing,2023-06-16 13:20:28,NaN,714000.0,2.0,2.0,HAMPTONS WEST CONDO,1800.0,NaN,1984.0,...,Active,,,https://www.redfin.com/FL/Aventura/20281-E-Cou...,MARMLS,A11343200,N,Y,25.965506,-80.125855
367240,MLS Listing,2023-06-16 13:20:28,NaN,499000.0,2.0,2.0,Pebble Beach,1800.0,11473.0,2002.0,...,Active,,,https://www.redfin.com/TX/Lakehills/918-Pebble...,Sabor,1649294,N,Y,29.584477,-98.928766
436674,MLS Listing,2023-06-16 13:20:28,NaN,599000.0,2.0,2.0,North Registry,1800.0,69130.0,2022.0,...,Active,,,https://www.redfin.com/MA/North-Adams/825-Moha...,BCMLS,237858,N,Y,42.692028,-73.085696
264621,MLS Listing,2023-06-16 13:20:28,NaN,279000.0,2.0,2.0,Edelweiss on the Lake,1800.0,NaN,1996.0,...,Active,,,https://www.redfin.com/IL/Palos-Park/8-Saint-M...,MRED,11799789,N,Y,41.677641,-87.817323
65432,MLS Listing,2023-06-16 13:20:28,NaN,250000.0,2.0,2.0,682 - Chino Hills,1800.0,NaN,1978.0,...,Active,,,https://www.redfin.com/CA/Chino-Hills/15111-Pi...,CRMLS,TR23021111,N,Y,33.978066,-117.711473


In [246]:
input_t

array([[2.02020202e-02, 3.00000000e-04, 8.64817406e-05, 2.28832952e+00]])

In [247]:
def loss_function(index,input,output):
    lst=[]
    
    for i in range(0,3):
        loss=np.abs((input[0][i]-output[index][i]))
        
        lst.append(loss)
        
    return np.mean(lst)

In [248]:
for index in indices[0][0:5]:
    vals=[]
    val=loss_function(index,input_t,df_NN)
    vals.append(val)
    
val_avg=np.mean(vals)

In [241]:
val_avg

0.003368381434391086

In [249]:
val_avg

0.0

In [229]:
df_full.loc[indices[0][0]]

ADDRESS                      1103 Fitzgerald Ct
CITY                                Summerville
STATE OR PROVINCE                            SC
ZIP OR POSTAL CODE                        29485
PROPERTY TYPE         Single Family Residential
PRICE                                  503805.0
BEDS                                        5.0
BATHS                                       3.0
SQUARE FEET                              2226.0
YEAR BUILT                               2748.0
DAYS ON MARKET                             20.0
LATITUDE                                32.9431
LONGITUDE                              -80.2772
Name: 281733, dtype: object

In [278]:
all=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')]

state=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')]

city=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')&
               (df_full['CITY']=='Houston')]

zip=df_full[(df_full['PROPERTY TYPE']=='Single Family Residential')&
               (df_full['STATE OR PROVINCE']=='TX')&
               (df_full['CITY']=='Houston')&
               (df_full['ZIP OR POSTAL CODE']=='77004')]

slices=[all,state,city,zip]

In [279]:
def loss_function(index,input,output):
    lst=[]
    
    for i in range(0,3):
        loss=np.abs((input[0][i]-output[index][i]))
        
        lst.append(loss)
        
    return np.mean(lst)
        
    


In [280]:
# OPTIMUM WEIGHT FOR YEAR BUILT SEARCH

# Beware, could take a VERY long time

results={} # Dict of dicts of lists of tuples

for i,slice in enumerate(slices):

    str=""
            
    if i==0:
        str='all'
        
    elif i==1:
        str='state'
        
    elif i==2:
        str='city'
        
    else:
        str='zip'

    results[str]={}
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('minmax_scaler',MinMaxScaler(),['BEDS','BATHS','SQUARE FEET'])
    ])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
    
    slice_np=pipeline.fit_transform(slice[['BEDS','BATHS','SQUARE FEET']])
    
    for year in [2000,2020,1900]:
        
        input_df = pd.DataFrame({'BEDS':[3],'BATHS':[3],'SQUARE FEET':[2600]})
        input_df

        input_t=pipeline.transform(input_df)
        input_t
        
        dct={"weights":[],"loss":[]}

        for w in np.linspace(0,1,21):
            
            def weighted_euclidean(x:np.array,y:np.array):
                weights=np.array([1,1,1,w])
                
                return euclidean(x,y,weights)
            
            knn=NearestNeighbors(n_neighbors=slice_np.shape[0],metric=weighted_euclidean)
            knn.fit(slice_np)

            distances, indices = knn.kneighbors(input_t)
            
            for index in indices[0][0:5]:
                vals=[]
                val=loss_function(index,input_t,slice_np)
                vals.append(val)
                
            val_avg=np.mean(vals)
                
            dct['weights'].append(w)
            dct['loss'].append(val_avg)
                
                
        results[str][year]=dct
            
        
                
                
            

            
        


In [284]:

results_df=pd.DataFrame(results['city'][1900])


    

In [285]:
results_df

,weights,loss
0,0.00,0.000652
1,0.05,0.018803
2,0.10,0.021078
3,0.15,0.021078
4,0.20,0.021078
5,0.25,0.020951
6,0.30,0.034936
7,0.35,0.038553
8,0.40,0.023034
9,0.45,0.023034


,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE
349423,3866 Wichita St,Houston,TX,77004,Single Family Residential,320000.0,3.0,2.0,2364.0,1970.0,20.0,29.712865,-95.351781
349424,2010 Blodgett St Unit H,Houston,TX,77004,Single Family Residential,379000.0,2.0,2.0,1799.0,2007.0,1.0,29.726870,-95.375840
349426,4909 La Branch St,Houston,TX,77004,Single Family Residential,489000.0,3.0,3.5,2415.0,2005.0,1.0,29.727276,-95.381520
349429,2705 Anita St,Houston,TX,77004,Single Family Residential,699900.0,4.0,3.5,2626.0,2023.0,3.0,29.734106,-95.361277
349430,1400 Dennis St,Houston,TX,77004,Single Family Residential,425000.0,4.0,2.0,1592.0,2000.0,3.0,29.742313,-95.370961
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349594,3505 Hutchins St,Houston,TX,77004,Single Family Residential,449000.0,3.0,3.5,2547.0,2015.0,279.0,29.733206,-95.368318
349595,2516 Saint Emanuel St,Houston,TX,77004,Single Family Residential,675000.0,3.0,2.5,2511.0,2022.0,303.0,29.740371,-95.364486
349596,3409 N Macgregor Way,Houston,TX,77004,Single Family Residential,775000.0,3.0,5.0,4784.0,1950.0,550.0,29.711009,-95.365352
349597,2201 Ruth St,Houston,TX,77004,Single Family Residential,777999.0,5.0,4.5,3500.0,1930.0,662.0,29.726790,-95.373365



### Warning: Regarding the Nearest Neighbors algorithms,if two neighbors and have identical distances but different labels, the result will depend on the ordering of the training data.

In [108]:
df_slice=df_full[df_full['ZIP OR POSTAL CODE']=='78705']

In [109]:
df_slice

,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PROPERTY TYPE,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,LATITUDE,LONGITUDE


In [110]:
picks=[]

for i in indices[0]:
    try:
        pick=df_slice.loc[i]
        picks.append(pick)
    except:
        pass
    
    if len(picks)==3:
        break

In [111]:
picks[0]

IndexError: list index out of range

In [91]:
picks[1]

ADDRESS                   2501 Truxillo St #6
CITY                                  Houston
STATE OR PROVINCE                          TX
ZIP OR POSTAL CODE                      77004
PROPERTY TYPE         Multi-Family (2-4 Unit)
PRICE                                780000.0
BEDS                                      9.0
BATHS                                     1.0
SQUARE FEET                            5000.0
YEAR BUILT                             1930.0
DAYS ON MARKET                           62.0
LATITUDE                             29.72961
LONGITUDE                          -95.367364
Name: 349536, dtype: object

In [71]:
df_full.iloc[349519]

ADDRESS                          3342 Reeves St
CITY                                    Houston
STATE OR PROVINCE                            TX
ZIP OR POSTAL CODE                        77004
PROPERTY TYPE         Single Family Residential
PRICE                                  250000.0
BEDS                                        3.0
BATHS                                       2.0
SQUARE FEET                              1104.0
YEAR BUILT                               2002.0
DAYS ON MARKET                             49.0
LATITUDE                              29.727252
LONGITUDE                            -95.355255
Name: 349519, dtype: object

In [298]:
input_t

array([[3.03030303e-02, 3.00000000e-04, 8.64817406e-05, 5.72082380e-01]])